In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
import PyPDF2

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

pdf_file = 'RAG_Chatbot_QA_Guide.pdf'
pdf_reader = PyPDF2.PdfReader(pdf_file)

qa_pairs = []
current_question = None
current_answer = []

for page in pdf_reader.pages:
    text = page.extract_text()
    if text:
        lines = text.split('\n')
        for line in lines:
            if line.startswith("Q: "):
                
                if current_question and current_answer:
                    qa_pairs.append((current_question, ' '.join(current_answer).strip()))
        
                current_question = line[3:].strip()
                current_answer = []
            elif line.startswith("A: "):
                
                current_answer.append(line[3:].strip())
            else:
        
                if current_answer:
                    current_answer.append(line.strip())

if current_question and current_answer:
    qa_pairs.append((current_question, ' '.join(current_answer).strip()))

questions = [qa[0] for qa in qa_pairs]
answers = [qa[1] for qa in qa_pairs]
question_embeddings = embedding_model.encode(questions, convert_to_tensor=True)

def find_answer(user_question):
    user_embedding = embedding_model.encode(user_question, convert_to_tensor=True)
    
    similarities = util.cos_sim(user_embedding, question_embeddings)[0]

    best_match_idx = torch.argmax(similarities).item()
    return answers[best_match_idx]


def chatbot():
    print("Chatbot Initialized! Type 'exit' to quit.")
    while True:
        user_question = input("You: ")
        if user_question.lower() == 'exit':
            print("Goodbye!")
            break
        answer = find_answer(user_question)
        print("Bot:", answer)

chatbot()
